### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Open Subtitles/Lesson Deployment2/ReDe"

# parameter
file = "Twogram_With_Word_List_Pair_Cross_Check"
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Twogram_With_28_And_172_Word_Pair"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6
sample_num = 10

In [3]:
#path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"
#
#Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [7]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [8]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [9]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [10]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram,frequency
0,ne oldu,322758
1,tamam mı,264540
2,öyle mi,263184
3,o zaman,213235
4,her şey,210860
5,çok güzel,157881
6,bir dakika,141865
7,çok iyi,131311
8,daha iyi,120626
9,sorun değil,118971


In [11]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
        else:
            pass

In [12]:
len(search_list)

36

In [13]:
#search_list

In [14]:
df_search_result = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,ne oldu,917.007,918.594,peki sonra ne oldu hocam,PyW7tKbYl1I
1,ne oldu,79.320,81.520,ne oldu sana sen iyi misin,Q9bo4mKKszc
2,ne oldu,47.430,60.010,ne oldu ki,F8kKnhi1_Zo
3,ne oldu,3080.851,3082.749,zeynep koluna ne oldu,CpUnvHn0xcI
4,ne oldu,4996.377,5001.721,sana inandım sana güvendim ne oldu,Qs1I7HxXYlg
...,...,...,...,...,...
355,ne ile,8093.100,8095.660,kimi ne ile tehdit edersen et,bIH7u90sJ3Q
356,ne ile,2768.655,2774.137,sen ne ile doldurur isen onun ile dolar,mE9PWvDSZwo
357,ne ile,5031.748,5032.975,ne ile ilgili,wKnBkKSY6zI
358,ne ile,189.359,195.620,tamir ekipmanıyla tamiri yapılır tamir ekipman...,Bp9Myhl1Mps


In [15]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,ne oldu,917.668250,918.263375,peki sonra ne oldu hocam,PyW7tKbYl1I
1,ne oldu,79.320000,79.996923,ne oldu sana sen iyi misin,Q9bo4mKKszc
2,ne oldu,47.430000,57.494000,ne oldu ki,F8kKnhi1_Zo
3,ne oldu,3082.025952,3082.749000,zeynep koluna ne oldu,CpUnvHn0xcI
4,ne oldu,5000.463588,5001.721000,sana inandım sana güvendim ne oldu,Qs1I7HxXYlg
...,...,...,...,...,...
355,ne ile,8093.453103,8094.159310,kimi ne ile tehdit edersen et,bIH7u90sJ3Q
356,ne ile,2769.076692,2770.201205,sen ne ile doldurur isen onun ile dolar,mE9PWvDSZwo
357,ne ile,5031.748000,5032.408692,ne ile ilgili,wKnBkKSY6zI
358,ne ile,194.692444,195.310815,tamir ekipmanıyla tamiri yapılır tamir ekipman...,Bp9Myhl1Mps


In [16]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,ne oldu,917.068250,918.863375,peki sonra ne oldu hocam,PyW7tKbYl1I
1,ne oldu,78.720000,80.596923,ne oldu sana sen iyi misin,Q9bo4mKKszc
2,ne oldu,46.830000,58.094000,ne oldu ki,F8kKnhi1_Zo
3,ne oldu,3081.425952,3083.349000,zeynep koluna ne oldu,CpUnvHn0xcI
4,ne oldu,4999.863588,5002.321000,sana inandım sana güvendim ne oldu,Qs1I7HxXYlg
...,...,...,...,...,...
355,ne ile,8092.853103,8094.759310,kimi ne ile tehdit edersen et,bIH7u90sJ3Q
356,ne ile,2768.476692,2770.801205,sen ne ile doldurur isen onun ile dolar,mE9PWvDSZwo
357,ne ile,5031.148000,5033.008692,ne ile ilgili,wKnBkKSY6zI
358,ne ile,194.092444,195.910815,tamir ekipmanıyla tamiri yapılır tamir ekipman...,Bp9Myhl1Mps


In [17]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,ne oldu,917,919,peki sonra ne oldu hocam,PyW7tKbYl1I
1,ne oldu,79,81,ne oldu sana sen iyi misin,Q9bo4mKKszc
2,ne oldu,47,58,ne oldu ki,F8kKnhi1_Zo
3,ne oldu,3081,3083,zeynep koluna ne oldu,CpUnvHn0xcI
4,ne oldu,5000,5002,sana inandım sana güvendim ne oldu,Qs1I7HxXYlg
...,...,...,...,...,...
355,ne ile,8093,8095,kimi ne ile tehdit edersen et,bIH7u90sJ3Q
356,ne ile,2768,2771,sen ne ile doldurur isen onun ile dolar,mE9PWvDSZwo
357,ne ile,5031,5033,ne ile ilgili,wKnBkKSY6zI
358,ne ile,194,196,tamir ekipmanıyla tamiri yapılır tamir ekipman...,Bp9Myhl1Mps


In [18]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,ne oldu,917,919,peki sonra ne oldu hocam,PyW7tKbYl1I,https://www.youtube.com/watch?v=PyW7tKbYl1I&t=...
1,ne oldu,79,81,ne oldu sana sen iyi misin,Q9bo4mKKszc,https://www.youtube.com/watch?v=Q9bo4mKKszc&t=79s
2,ne oldu,47,58,ne oldu ki,F8kKnhi1_Zo,https://www.youtube.com/watch?v=F8kKnhi1_Zo&t=47s
3,ne oldu,3081,3083,zeynep koluna ne oldu,CpUnvHn0xcI,https://www.youtube.com/watch?v=CpUnvHn0xcI&t=...
4,ne oldu,5000,5002,sana inandım sana güvendim ne oldu,Qs1I7HxXYlg,https://www.youtube.com/watch?v=Qs1I7HxXYlg&t=...
...,...,...,...,...,...,...
355,ne ile,8093,8095,kimi ne ile tehdit edersen et,bIH7u90sJ3Q,https://www.youtube.com/watch?v=bIH7u90sJ3Q&t=...
356,ne ile,2768,2771,sen ne ile doldurur isen onun ile dolar,mE9PWvDSZwo,https://www.youtube.com/watch?v=mE9PWvDSZwo&t=...
357,ne ile,5031,5033,ne ile ilgili,wKnBkKSY6zI,https://www.youtube.com/watch?v=wKnBkKSY6zI&t=...
358,ne ile,194,196,tamir ekipmanıyla tamiri yapılır tamir ekipman...,Bp9Myhl1Mps,https://www.youtube.com/watch?v=Bp9Myhl1Mps&t=...


In [19]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

#### Copy Move And Delete

In [18]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{file_ext}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx")
output_file

['Turkish_Spanish_2 gram hybrid_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [19]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [20]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass